In [1]:
import os
from dotenv import load_dotenv
from langchain_neo4j import Neo4jGraph

load_dotenv()

#LangChain 도구 활용 - DB연결 객체 초기화
graph = Neo4jGraph(
    url = os.getenv("NEO4J_URI"),
    username = os.getenv("NEO4J_USERNAME"),
    password = os.getenv("NEO4J_PASSWORD"),
    enhanced_schema=True, # 확장 스키마 출력 설정 (그래프의 노드/관계 타입뿐만 아니라 속성 정보와 데이터 샘플까지 추출)
                          # 사용 목적 : LLM이 Cypher 쿼리를 생성할 때 정확도를 높이기 위함
)

In [3]:
#테스트 쿼리 실행
cypher_query = """
MATCH (n:Movie)
RETURN COUNT (n) AS Movie_Count
"""

graph.query(cypher_query)

[{'Movie_Count': 4803}]

### 2.1 백터 검색 (Semantic Search)

#### 1) Graph DB 초기화

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_neo4j import Neo4jVector
from langchain_google_genai import GoogleGenerativeAIEmbeddings

#임베딩 모델 초기화
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

#Neo4j 데이터베이스에 이미 생성된 벡터 인덱스에 연결하는 Neo4jVector 인스턴스 생성
graph_db = Neo4jVector.from_existing_index(
    embeddings,
    url = os.getenv("NEO4J_URI"),
    username = os.getenv("NEO4J_USERNAME"),
    password = os.getenv("NEO4J_PASSWORD"),
    index_name = "movie_content_embeddings", # 사용할 벡터 인덱스 이름 (이미 Neo4j에 생성되어 있어야 함)
    text_node_property = "overview", # 텍스트 검색 시 반환할 노드의 속성 (영화 개요)
)

#### 2) 벡터 검색 (유사도 기준)

In [17]:
# 한국어로 된 자연어 쿼리를 사용하여 의미적으로 유사한 영화 검색
query = "미국을 배경으로 동물들이 주인공인 애니메이션 영화를 찾아주세요."

#유사도 검색 수행
similar_docs = graph_db.similarity_search_with_score(
    query,
    k=5, #유사도 상위 5개 문서 검색
    return_embeddings = False #임베딩 반환 안함 (결과 간소화)
)

# 각 문서와 해당 유사도 점수를 함께 표시
for doc, score in similar_docs:
    print(f"줄거리: {doc.page_content[:100]}...., 유사도: {score}")
    print("영화 제목:", doc.metadata.get("title"))
    print("-"*50)

줄거리: This video shows how the foreign policy interests of American political elites-working in combinatio...., 유사도: 0.7543048858642578
영화 제목: Peace, Propaganda & the Promised Land
--------------------------------------------------
줄거리: On the hottest day in 50 years, a serious fire incident happened to a busy commercial tower, a gaggl...., 유사도: 0.7539863586425781
영화 제목: Out of Inferno
--------------------------------------------------
줄거리: In an unprecedented and candid series of interviews, six former heads of the Shin Bet — Israel's int...., 유사도: 0.7537875175476074
영화 제목: The Gatekeepers
--------------------------------------------------
줄거리: Gang-du is a dim-witted man working at his father's tiny snack bar near the Han River. One day, Gang...., 유사도: 0.7532477378845215
영화 제목: The Host
--------------------------------------------------
줄거리: With no clue how he came to be imprisoned, drugged and tortured for 15 years, a desperate businessma...., 유사도: 0.7481241226196289
영화 제목: Oldboy
-

In [18]:
doc.metadata

{'rating': 8.0,
 'runtime': 120,
 'tagline': '15 years of imprisonment, five days of vengeance',
 'title': 'Oldboy',
 'released': '2003-01-01'}

#### 3) MMR 알고리즘을 활용한 다양성 검색

In [20]:
#MMR 알고리즘 : 질문과 가장 관련이 높으면서도, 이미 보여준 결과와는 중복되지 않는 "새로운 정보"를 우선순위에 두는 방식

#벡터 검색 쿼리 실행
query = "동물들이 주인공인 애니메이션 영화를 찾아주세요."

similar_docs = graph_db.max_marginal_relevance_search(
    query = query,
    k=5,
    fetch_k=20, # 더 많은 후보를 가져옴
    lambda_mult = 0.5 # 0에 가까울 수록 다양성 높음 (0.5는 적당한 균형을 의미함)
)

for doc in similar_docs:
    print(f"문서 내용: {doc.page_content[:100]}...")
    print("영화 제목:", doc.metadata.get("title"))
    print("-"*50)

문서 내용: Gang-du is a dim-witted man working at his father's tiny snack bar near the Han River. One day, Gang...
영화 제목: The Host
--------------------------------------------------
문서 내용: This video shows how the foreign policy interests of American political elites-working in combinatio...
영화 제목: Peace, Propaganda & the Promised Land
--------------------------------------------------
문서 내용: A word for word depiction of the life of Jesus Christ from the Good News Translation Bible as record...
영화 제목: The Visual Bible: The Gospel of John
--------------------------------------------------
문서 내용: Zohre's shoes are gone; her older brother Ali lost them. They are poor, there are no shoes for Zohre...
영화 제목: Children of Heaven
--------------------------------------------------
문서 내용: This film tells the story of professor Uehida Hyakken-sama (1889-1971), in Gotemba, around the forti...
영화 제목: Madadayo
--------------------------------------------------


#### 4) 그래프 경로를 활용한 검색 확장

In [23]:
graph_db.query("SHOW INDEXES")

[{'id': 2,
  'name': 'genre_namne_unique',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'RANGE',
  'entityType': 'NODE',
  'labelsOrTypes': ['Genre'],
  'properties': ['name'],
  'indexProvider': 'range-1.0',
  'owningConstraint': 'genre_namne_unique',
  'lastRead': neo4j.time.DateTime(2025, 12, 25, 23, 35, 49, 43000000, tzinfo=<UTC>),
  'readCount': 102},
 {'id': 11,
  'name': 'movie_content_embeddings',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Movie'],
  'properties': ['content_embedding'],
  'indexProvider': 'vector-3.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2025, 12, 26, 1, 40, 46, 981000000, tzinfo=<UTC>),
  'readCount': 20},
 {'id': 4,
  'name': 'movie_id_unique',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'RANGE',
  'entityType': 'NODE',
  'labelsOrTypes': ['Movie'],
  'properties': ['id'],
  'indexProvider': 'range-1.0',
  'owningConstraint': 'mo

In [27]:
def find_movies_and_actors_filography(query, graph_db, k=5):
    """
    특정 영화에 출연한 배우들의 필모그래피를 찾는 함수
    
    Args:
        query : 검색 쿼리 (예: "2차 세계대전 영화")
        graph_db : 벡터 저장소 객체
        k : 검색할 영화 수
    
    Returns :
        영화와 배우 정보가 계층적으로 정리된 결과
    """

    #1단계 : 벡터 검색으로 영화 검색
    results = graph_db.similarity_search(query, k=k)
    print(f"벡터스토어 검색된 영화 수: {len(results)}")

    #검색된 영화 제목 추출
    movie_titles = []
    for doc in results :
        title = doc.metadata.get("title")
        if title : 
            movie_titles.append(title)
    
    #제목이 없으면 빈 결과 반환
    if not movie_titles:
        return {"movies":[], "message":"검색된 영화가 없습니다."}
    
    #Neo4j 그래프 객체 생성
    graph_db = Neo4jVector.from_existing_index(
        embeddings,
        index_name="movie_content_embeddings",
        url = os.getenv("NEO4J_URI"),
        username = os.getenv("NEO4J_USERNAME"),
        password = os.getenv("NEO4J_PASSWORD"),
    )

    #2단계: 영화와 출연 배우 정보 가져오기
    movies_actors_query="""
    //영화 제목 중 하나라도 포함하는 영화 필터링
    MATCH (movie: Movie)
    WHERE ANY(title IN $movie_titles WHERE movie.title CONTAINS title)

    //영화에 출연한 배우가 없을 수도 있으므로 OPTIONAL 사용
    OPTIONAL MATCH (movie)<-[acted:ACTED_IN]-(actor:Person)
    
    //각 영화별로 출연 배우들을 배열로 수집
    WITH movie, collect(actor) as actors

    //영화 정보와 배우 정보를 함께 반환
    RETURN
        movie.title as movie_title,
        movie.released as release_date,
        movie.rating as rating,
        movie.overview as overview,
        [actor IN actors | actor.name] as actor_names, //배우 이름 배열
        [actor IN actors | {name: actor.name, id: id(actor)}] as actors // 배우 정보 배열
    """

    search_movies = graph.query(movies_actors_query, params={"movie_titles": movie_titles})

    #배우 ID 목록 추출
    actor_ids=[]
    for movie in search_movies:
        for actor in movie.get("actors",[]):
            if "id" in actor:
                actor_ids.append(actor["id"])
    
    #중복 제거 (동일한 배우가 여러 영화에 출연할 수 있음)
    actor_ids = list(set(actor_ids))

    #배우가 없으면 영화 정보만 반환
    if not actor_ids:
        return {
            "movies": search_movies,
            "message": "검색된 영화에서 배우 정보를 찾을 수 없습니다."
        }
    
    #3단계 : 각 배우의 필모그래피 가져오기
    actor_filmography_query="""
    //특정 ID를 가진 배우 노드 찾기
    MATCH (actor:Person)
    WHERE id(actor) IN $actor_ids

    //배우가 출연한 영화 찾기
    MATCH (actor)-[:ACTED_IN]->(movie:Movie)

    //원래 검색된 영화는 제외 (중복방지)
    WHERE NOT movie.title IN $movie_titles

    WITH actor.name as actor_name, collect({
        title: movie.title,
        released : movie.released,
        rating : movie.rating
    }) as other_movies // 배우별로 출연 영화 정보 수집
    RETURN actor_name, other_movies // 배우 이름과 출연 영화 목록 반환
    ORDER BY actor_name // 배우 이름 알파벳 순으로 정렬
    
    """

    actor_filmographis = graph.query(
        actor_filmography_query,
        params={"actor_ids": actor_ids, "movie_titles": movie_titles}
    )

    #결과 데이터 구조화 - 영화 정보와 배우 필모그래피를 함께 반환
    result = {
        "search_movies": search_movies,
        "actor_filographies": actor_filmographis
    }

    return result

#검색 쿼리 실행
query ="2차 세계대전을 배경으로 군인들의 활약상을 그린 영화를 찾아주세요."
search_results = find_movies_and_actors_filography(
    query=query,
    graph_db = graph_db,
    k=3
)

#결과 출력 - 영화정보
for movie in search_results["search_movies"]:
    print(f"\n제목: {movie['movie_title']} ({movie.get('release_date', '연도 정보 없음')})")
    print(f"평점: {movie.get('rating','평점 정보 없음')}")
    print(f"출연 배우: {', '.join(movie.get('actor_names', []))}")
    if movie.get('overview'):
        print(f"줄거리: {movie['overview'][:150]}..." if len(movie['overview']) >150 else movie['overview'])
    print("-"*50)

벡터스토어 검색된 영화 수: 3


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function. ('id' has been replaced by 'elementId or consider using an application-generated id')} {position: line: 19, column: 51, offset: 598} for query: '\n    //영화 제목 중 하나라도 포함하는 영화 필터링\n    MATCH (movie: Movie)\n    WHERE ANY(title IN $movie_titles WHERE movie.title CONTAINS title)\n\n    //영화에 출연한 배우가 없을 수도 있으므로 OPTIONAL 사용\n    OPTIONAL MATCH (movie)<-[acted:ACTED_IN]-(actor:Person)\n\n    //각 영화별로 출연 배우들을 배열로 수집\n    WITH movie, collect(actor) as actors\n\n    //영화 정보와 배우 정보를 함께 반환\n    RETURN\n        movie.title as movie_title,\n        movie.released as release_date,\n        movie.rating as rating,\n        movie.overview as overview,\n        [actor IN actors | actor.name] as actor_names, //배우 이름 배열\n        [a


제목: Mission: Impossible II (2000-05-24)
평점: 5.9
출연 배우: Tom Cruise, Ving Rhames, Thandie Newton, Richard Roxburgh, Dougray Scott
줄거리: With computer genius Luther Stickell at his side and a beautiful thief on his mind, agent Ethan Hunt races across Australia and Spain to stop a former...
--------------------------------------------------

제목: Mission: Impossible III (2006-05-03)
평점: 6.5
출연 배우: Tom Cruise, Ving Rhames, Philip Seymour Hoffman, Billy Crudup, Jonathan Rhys Meyers
줄거리: Retired from active duty to train new IMF agents, Ethan Hunt is called back into action to confront sadistic arms dealer, Owen Davian. Hunt must try t...
--------------------------------------------------

제목: The Grudge 2 (2006-10-13)
평점: 5.2
출연 배우: Jennifer Beals, Sarah Michelle Gellar, Amber Tamblyn, Arielle Kebbel, Edison Chen
줄거리: In Tokyo, a young woman is exposed to the same mysterious curse that afflicted her sister. The supernatural force, which fills a person with rage befo...
-----------------------

### 2.2 LCEL사용하여 RAG 체인 구성
#### 1) Graph DB기반 Retriever 활용

In [28]:
retriever = graph_db.as_retriever(
    search_kwargs={"k":5}, # 검색할 문서 수
)

query = "2차 세계대전을 배경으로 군인들의 활약상을 그린 영화를 찾아주세요."
results = retriever.invoke(query)
for result in results:
    print(result.page_content[:100])
    print(result.metadata['title'])
    print("-"*50)

In Tokyo, a young woman is exposed to the same mysterious curse that afflicted her sister. The super
The Grudge 2
--------------------------------------------------
With computer genius Luther Stickell at his side and a beautiful thief on his mind, agent Ethan Hunt
Mission: Impossible II
--------------------------------------------------
Just as Dan and Kristi welcome a newborn baby into their home, a demonic presence begins terrorizing
Paranormal Activity 2
--------------------------------------------------
Rachel Keller must prevent evil Samara from taking possession of her son's soul.
The Ring Two
--------------------------------------------------
It's vacation time for Carter as he finds himself alongside Lee in Hong Kong wishing for more excite
Rush Hour 2
--------------------------------------------------


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough